In [1]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

In [2]:
#Reading data dsource csv
df = pd.read_csv('../cleaned_data.csv', index_col=0)

In [3]:
df.head()

,target,cap-shape_c,cap-shape_f,cap-shape_k,cap-shape_s,cap-shape_x,cap-surface_g,cap-surface_s,cap-surface_y,cap-color_c,...,population_n,population_s,population_v,population_y,habitat_g,habitat_l,habitat_m,habitat_p,habitat_u,habitat_w
0,p,0,0,0,0,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
1,e,0,0,0,0,1,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
2,e,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,1,0,0,0
3,p,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
4,e,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [4]:
X = df.drop('target', axis=1)

In [5]:
y = df['target']

In [6]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

Learning by Random Forest

In [7]:
random_tree_model = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

In [8]:
random_tree_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [9]:
y_pred = random_tree_model.predict(X_test)

In [10]:
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test,y_pred))

[[1293    0]
 [ 127 1018]]
              precision    recall  f1-score   support

           e       0.91      1.00      0.95      1293
           p       1.00      0.89      0.94      1145

   micro avg       0.95      0.95      0.95      2438
   macro avg       0.96      0.94      0.95      2438
weighted avg       0.95      0.95      0.95      2438



In [11]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(random_tree_model, X, y, cv = 10)
print(scores)

[0.6199262  0.88314883 0.98646986 0.98523985 0.96432964 0.94710947
 1.         1.         0.99506782 0.99506782]


In [12]:
scores.mean()

0.937635949725712